# Prophet - Predecir el valor del Café 


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Café, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [86]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

ERROR: Operation cancelled by user


In [87]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [88]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-04-19'
start_date = '2016-01-01'
# Descargar el dataframe
kc_df = yf.download('KC=F',start_date, today).reset_index()

kc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1831,2023-04-12,$188.90,$191.35,$187.60,$190.25,$190.25,27163
1832,2023-04-13,$190.00,$199.70,$189.35,$196.10,$196.10,35204
1833,2023-04-14,$195.65,$199.65,$192.85,$193.40,$193.40,11459
1834,2023-04-17,$194.00,$201.50,$193.50,$201.00,$201.00,9534
1835,2023-04-18,$201.00,$206.95,$198.35,$205.25,$205.25,9534


In [89]:
# Validamos que no hay datos vacíos
kc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [90]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = kc_df[["Date","Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names) 

In [91]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [92]:
df.tail()

,ds,y
1831,2023-04-12,$188.90
1832,2023-04-13,$190.00
1833,2023-04-14,$195.65
1834,2023-04-17,$194.00
1835,2023-04-18,$201.00


In [93]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Serie temporal precio de apertura del Café"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [94]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')#Hiperparámetro

In [95]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [96]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(365)
future.tail()

,ds
2196,2024-04-13
2197,2024-04-14
2198,2024-04-15
2199,2024-04-16
2200,2024-04-17


In [97]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future.head(10)

,ds
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
5,2016-01-11
6,2016-01-12
7,2016-01-13
8,2016-01-14
9,2016-01-15


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [98]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [99]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2092,2024-04-11,$126.34,$-14.57,$267.95
2093,2024-04-12,$125.97,$-10.81,$267.44
2094,2024-04-15,$125.07,$-15.93,$266.25
2095,2024-04-16,$125.03,$-15.91,$267.75
2096,2024-04-17,$124.80,$-13.78,$267.46


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [100]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [101]:
plot_components_plotly(m, forecast)